[](http://)

### Prepare main code

In [ ]:
!cp /kaggle/input/meta-learning-for-multi-task-multilingual/* /kaggle/working/ -r

In [ ]:
!mkdir data

### Prepare XTREME dataset

In [ ]:
!cp /kaggle/input/xtreme/* /kaggle/working/data -r

### Prepare PersianQA dataset

In [ ]:
!cp /kaggle/input/persianqa /kaggle/working/data/persianqa -r

In [ ]:
import json
from sklearn.model_selection import train_test_split

with open('/kaggle/working/data/persianqa/pqa_train.json', 'r') as train_file:
    data = json.load(train_file)
    train, dev = train_test_split(data['data'], test_size=0.1, random_state=42)
    
    trainString = json.dumps({"data": train})
    devString = json.dumps({"data": dev})
   
with open('/kaggle/working/data/persianqa/pqa_train.json', 'w') as train_file:
    train_file.write(trainString)
    
with open('/kaggle/working/data/persianqa/pqa_dev.json', 'w') as dev_file:
    dev_file.write(devString)

### Prepare FarsTail dataset

In [ ]:
!cp /kaggle/input/farstail/data /kaggle/working/data/farstail -r

In [ ]:
import pandas as pd

labels_dict = {'e': 'entailment', 'c': 'contradiction', 'n': 'neutral'}

def convertFarsTailData(filename):
    df = pd.read_csv(f'/kaggle/working/data/farstail/{filename}', sep='\t')
    df = df.apply(lambda row: [row[0], row[1], labels_dict[row[2]]], axis=1, result_type='expand')
    df.to_csv(f'/kaggle/working/data/farstail/{filename}', sep='\t', header=None, index=False)

convertFarsTailData('Train-word.csv')
convertFarsTailData('Test-word.csv')
convertFarsTailData('Val-word.csv')

### Install required packages

In [ ]:
!conda install -q -c conda-forge pickle5 -y

# !pip3 install pickle5
import pickle5 as pickle

In [ ]:
!pip install -q transformers==2.11.0

### Train on the source tasks & languages

In [ ]:
!python main.py --meta_tasks qa_fa,qa_en,sc_en --temp 5.0

### Zero-shot test

In [ ]:
!python3 zeroshot.py --load saved/model_sc.pt --task sc_fa

### Fine-tune on the target task

In [ ]:
!python3 baseline.py --load saved/model_sc.pt --task sc_fa --epochs 5

### Train on the source and target tasks and languages

In [ ]:
!python main.py --meta_tasks qa_fa,sc_fa,qa_en,sc_en --temp 5.0

In [ ]:
!python3 zeroshot.py --load saved/model_sc.pt --task sc_fa

In [ ]:
!python3 baseline.py --load saved/model_sc.pt --task sc_fa --epochs 5